In [16]:
import MySQLdb
import sshtunnel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from scipy.stats import pearsonr
from ast import literal_eval
import matplotlib.pyplot as plt

In [17]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password='midgeq-xikFah-gukpu7',
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd='david2202087',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        query = "SELECT * FROM condition_1;"  # Replace 'your_table_name' with the actual table name
        df = pd.read_sql(query, connection)
        print("Data read finished")
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished


### Attention Pass Check

In [18]:
id_attention_failed = df[df['attention_passed'] == 0]['participantId'].tolist()
print(f"{len(id_attention_failed)} participants failed attention passed: {id_attention_failed}")
columns_to_drop = ['assignmentId', 'projectId', 'attention_passed', 'total_time', 'identity_choices', 'ideologies', 'submit_time']
df.drop(columns_to_drop, inplace=True, axis=1)
df.head()
df.set_index('participantId', inplace=True)
df.head()

0 participants failed attention passed: []


,bot_detected,non_ideology_answers,additional_answers,reason
participantId,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[{""answers"": [-100, 1.1, -100], ""time_to_answe...","[[0.6, 0.2, -0.2]]",None
0B8D09A73ED140CF84C5320DAE27F350,50,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, 2, 1]]",None
0B97FA950789450AA070261124D188E7,33,"[{""answers"": [-100, 1.666666666666667, -100], ...","[[0.1, -0.1, 0.3]]",I just assume that they are bots based on how ...
0BA66FB40A324FCDB1E3E79F2CE76035,33,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, -2, 1]]",
111B9C01B7A24FB1BA760C980D409019,53,"[{""answers"": [-100, 1.8, -100], ""time_to_answe...","[[1, -0.7, 1]]",


### Bot detection

In [19]:
def get_bot_detected(detection):
    if detection >= 30 and detection <= 39 or detection % 10 == 3:
        return True
    return False

df['bot_detected'] = df['bot_detected'].apply(get_bot_detected)

In [20]:
bot_detected_ratio = df['bot_detected'].sum(axis=0) / len(df['bot_detected'])
print(f"There are {bot_detected_ratio:.2%} participants who detected bots")
print("\nReasons:")
for reason in df['reason'].tolist():
    if reason and reason != 'None':
        print(reason)
df.drop(['reason'], axis=1, inplace=True)

There are 30.00% participants who detected bots

Reasons:
I just assume that they are bots based on how the interactions played out.
I actually selected a blue collar worker simply because i was believing i was actually been matched.


In [21]:
df.head()

,bot_detected,non_ideology_answers,additional_answers
participantId,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[{""answers"": [-100, 1.1, -100], ""time_to_answe...","[[0.6, 0.2, -0.2]]"
0B8D09A73ED140CF84C5320DAE27F350,False,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, 2, 1]]"
0B97FA950789450AA070261124D188E7,True,"[{""answers"": [-100, 1.666666666666667, -100], ...","[[0.1, -0.1, 0.3]]"
0BA66FB40A324FCDB1E3E79F2CE76035,True,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, -2, 1]]"
111B9C01B7A24FB1BA760C980D409019,True,"[{""answers"": [-100, 1.8, -100], ""time_to_answe...","[[1, -0.7, 1]]"


### Compare distance between two cases
1. When human answers first, calculate the distance between his answer and the other two bots' following answers.
2. If a bot answers first, calculate the distance between his answer and human's following answer

Result should be that given each question, the distance in case 2 is smaller than that in case 1.

Calculate 3 thins for each question:
1. distance in case 1
2. distance in case 2
3. average answer of human if human answers first to examine the result of pilot_2

In [22]:
df['non_ideology_answers'] = df['non_ideology_answers'].apply(literal_eval)
each_answer = df.explode('non_ideology_answers')

In [32]:
each_answer['answers'] = each_answer.apply(lambda row: row['non_ideology_answers']['answers'], axis=1)
each_answer['idx_of_question'] = each_answer.apply(lambda row: row['non_ideology_answers']['idx_of_question'], axis=1)
each_answer['human_answers_first'] = each_answer.apply(lambda row: row['non_ideology_answers']['who_answers_first'] == 1, axis=1)
each_answer.drop(['non_ideology_answers'], axis=1, inplace=True)
each_answer.head()

,bot_detected,additional_answers,answers,idx_of_question,human_answers_first
participantId,,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[[0.6, 0.2, -0.2]]","[-100, 1.1, -100]",5,True
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[[0.6, 0.2, -0.2]]","[-100, -0.5, -1.6]",0,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[[0.6, 0.2, -0.2]]","[1.9, 0.6, -100]",1,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[[0.6, 0.2, -0.2]]","[-100, 1.0500000000000007, -100]",3,True
0AA37D5D01BA4B76A3FF64C0B1F49B70,False,"[[0.6, 0.2, -0.2]]","[-2.066666666666667, -1.5333333333333337, -100]",2,False


In [51]:
each_question = each_answer.groupby('idx_of_question')
human_index = 1

def calculate_average_human_answer(group):
    human_answers = [answers[human_index] for answers in group['answers']]
    print(human_answers)
    return sum(human_answers) / len(human_answers)

result = calculate_average_human_answer(each_question)

[participantId
0AA37D5D01BA4B76A3FF64C0B1F49B70    [-100, -0.5, -1.6]
0B8D09A73ED140CF84C5320DAE27F350       [-100, 1, -0.3]
0B97FA950789450AA070261124D188E7       [-100, 0, -2.9]
0BA66FB40A324FCDB1E3E79F2CE76035          [-100, 2, 1]
111B9C01B7A24FB1BA760C980D409019     [-100, -1.1, 2.7]
24BC3A44EAB247DD8A64EE8D7C02D0E5     [0.8, -2.1, -100]
364BA2EEA202439CA73EC72F6571CDBD    [-0.2, -1.5, -100]
3AC52CBC50AB48508580FCE92161CB52        [-100, 2, 2.8]
43525690125D44EE930AD12BBE9F05C4    [-100, -1.4, -2.2]
5F7A839198A043C59933778B2AA76ACC      [-100, 2.5, 2.9]
74AF200637384EC79071D83472997B5B     [-100, -0.9, 0.7]
7D0115E866E04B4DAEB395CF66EE6E93      [-100, -1, -100]
7D0CC43EFF454085A097E314118598BA    [-100, -2.7, -2.5]
9BA21584E852433B84023184E3D937F6       [-100, -3, 0.9]
A03E5FA85ABF4E88890A05D201D11D29     [-100, 1.9, -100]
BA13C9F8BDE84BF085C5025BB854177D       [-2.6, 2, -100]
CEC7A4B04C6A45F18B8573869B6F8D88      [-100, 2.1, 2.3]
F792153FC62D400CBC86DD79F13DA91A    [-100, -1.1, -

TypeError: unsupported operand type(s) for +: 'int' and 'list'